In [ ]:
import numpy as np
from astropy.table import Table
import pylab as plt

In [ ]:
IMAX = 32 # maximum integer wave number
DELTAK = 2. * np.pi / 10000.0 # wave number spacing in inverse pixels

In [ ]:
# functions to set up design matrices

def fourier_functions(xs, ys):
    n = len(xs)
    assert len(ys) == n
    fxs = np.zeros((n, IMAX * 2 + 2))
    fys = np.zeros((n, IMAX * 2 + 2))
    iis = np.zeros(IMAX * 2 + 2).astype(int)
    for i in range(IMAX+1):
        fxs[:, i * 2] = np.cos(i * DELTAK * xs)
        fys[:, i * 2] = np.cos(i * DELTAK * ys)
        iis[i * 2] = i
        fxs[:, i * 2 + 1] = np.sin((i + 1) * DELTAK * xs)
        fys[:, i * 2 + 1] = np.sin((i + 1) * DELTAK * ys)
        iis[i * 2 + 1] = i + 1
    return fxs, fys, iis

def design_matrix(xs, ys):
    fxs, fys, iis = fourier_functions(xs, ys)
    n, p = fxs.shape
    Xbig = (fxs[:, :, None] * fys[:, None, :]).reshape((n, p * p))
    i2plusj2 = (iis[:, None] ** 2 + iis[None, :] ** 2).reshape(p * p)
    return Xbig[:, i2plusj2 <= IMAX ** 2]

In [ ]:
data = Table.read("dxyPixels.csv", format='ascii.csv')

In [ ]:
plt.plot(data["x"], data["y"], "k.")

In [ ]:
xs = data["x"]
ys = data["y"]
dxs = data["dx"]
dys = data["dy"]

In [ ]:
xs.shape

In [ ]:
X = design_matrix(xs, ys)
X.shape

In [ ]:
n, p = X.shape

In [ ]:
np.random.seed(42)
rands = np.random.uniform(size=n)

In [ ]:
train = rands <= 0.8
test = rands > 0.8
print(np.sum(train), np.sum(test))

In [ ]:
beta_x, resids, rank, s = np.linalg.lstsq(X[train], dxs[train], rcond=None)
dxs_hat = X[test] @ beta_x
print(rank, min(s), max(s))

In [ ]:
print("original dx (test set) RMS:", np.sqrt(np.mean(dxs[test] ** 2)))
print("dx - dx_hat (test set) RMS:", np.sqrt(np.mean((dxs[test] - dxs_hat) ** 2)))
print("dx - dx_hat (test set) MAD:", np.sqrt(np.median((dxs[test] - dxs_hat) ** 2)))

In [ ]:
beta_y, resids, rank, s = np.linalg.lstsq(X[train], dys[train], rcond=None)
dys_hat = X[test] @ beta_y
print(rank, min(s), max(s))

In [ ]:
print("original dy (test set) RMS:", np.sqrt(np.mean(dys[test] ** 2)))
print("dy - dy_hat (test set) RMS:", np.sqrt(np.mean((dys[test] - dys_hat) ** 2)))
print("dy - dy_hat (test set) MAD:", np.sqrt(np.median((dys[test] - dys_hat) ** 2)))